In [195]:
embed_dim=100
G=4
#batch_size=80
number_of_negative_tokens = 5
n_epochs = 30
n_workers=64
queue_size = 64
padding_length=3
title_length_percentile=95
desc_length_percentile=90

import os, time, json,pandas as pd, numpy as np, re, string, random,sys
from collections import Counter,defaultdict
import Levenshtein


from keras.layers import Input,Lambda,Conv1D,GlobalMaxPool1D,concatenate,Dense,Dropout,PReLU,TimeDistributed,Embedding,Activation
from keras import Model
import keras.backend as K
import keras
from keras.utils import Sequence
import tensorflow as tf



df= pd.read_csv('yummly.csv')


# In[7]:


In [196]:
df.WHO.value_counts()

1    30405
2    26269
3    15468
4     8259
5     4050
0     3357
6     1413
7      192
Name: WHO, dtype: int64

In [197]:
pd.options.display.max_colwidth= 500
df.head(1)

,RECIPE,WHO
0,"toasted sesame oil#4.533333333333333,olive oil#53.099999999999994,nut butter#129.0,cider vinegar#58.60666666666667,honey#61.019999999999996,ground cayenne pepper#0.22499999999999998,lime wedges#33.5,fresh spinach#142.0,spearmint#13.0,spring onions#75.0",2


In [198]:
#!pip3 install python-levenshtein
#!pip3 install setuptools

# strip punctuation and combine ingredients into one word separated by a spae

df['RECIPE'] = df['RECIPE'].str.replace('\d+', '') #strip all digits
df['RECIPE'] = df['RECIPE'].str.replace(' ', '')
df['RECIPE'] = df['RECIPE'].str.replace('#.,', ' ')
df['RECIPE'] = df['RECIPE'].str.replace('#.', '')

#df['RECIPE'] = df['RECIPE'].str.replace(' ', '')
#df['RECIPE'] = df['RECIPE'].str.replace(' ', '')
#df['RECIPE'] = df['RECIPE'].str.replace('#', ' ')
#df['RECIPE'] = df['RECIPE'].str.replace(r'[\s^\w]', '')
#df['RECIPE'] = df['RECIPE'].str.replace(' ', '')
#df['RECIPE'] = df['RECIPE'].str.replace(',', ' ')
#df['RECIPE'] = df['RECIPE'].str.replace('\d', ' ')






In [199]:
df.head(12)

,RECIPE,WHO
0,toastedsesameoil oliveoil nutbutter cidervinegar honey groundcayennepepper limewedges freshspinach spearmint springonions,2
1,oliveoil cakeflour whiteonion choppedgarlic whitewine grapetomatoes chickenbouillon tomatopaste darkbrownsugar parsley,2
2,fineseasalt crackedblackpepper choppedgarlic whiteonion ice freshbasil freshoregano freshthymeleaves saltedbutter shreddedparmesancheese,1
3,unsaltedbutter shortening darkbrownsugar whitesugar hardboiledeggs extract cakeflour bakingsoda cinnamonsticks fineseasalt rolledoats goldenraisins,2
4,ice fineseasalt saltedbutter cakeflour crackedblackpepper sourcream,2
5,saltedbutter sweetpotatoes pineapple puremaplesyrup cinnamonsticks fineseasalt choppedwalnuts,3
6,hamburgerbuns pastasauce,2
7,coldmilk cakeflour oliveoil whitesugar fineseasalt meltedbutter,5
8,canolaoil whiteonion tofu garlicpowder cumin fineseasalt groundturmeric crackedblackpepper,3
9,whiteonion fineseasalt crackedblackpepper cakeflour saltedbutter coldmilk whitesugar eggyolks,5


In [200]:
df['WHO'] = df['WHO'].replace({0:'zero', 1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five', 6: 'six', 7: 'seven'})

In [201]:
pd.options.display.max_colwidth= 500
df.head(4)

,RECIPE,WHO
0,toastedsesameoil oliveoil nutbutter cidervinegar honey groundcayennepepper limewedges freshspinach spearmint springonions,two
1,oliveoil cakeflour whiteonion choppedgarlic whitewine grapetomatoes chickenbouillon tomatopaste darkbrownsugar parsley,two
2,fineseasalt crackedblackpepper choppedgarlic whiteonion ice freshbasil freshoregano freshthymeleaves saltedbutter shreddedparmesancheese,one
3,unsaltedbutter shortening darkbrownsugar whitesugar hardboiledeggs extract cakeflour bakingsoda cinnamonsticks fineseasalt rolledoats goldenraisins,two


In [202]:
from sklearn.model_selection import train_test_split
#(X_train, X_test) = train_test_split(df2,test_size=0.20)


#with tf.device('/gpu:0'):
X_train, X_validation, X_test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])

descTokenCounts = pd.Series(' '.join(df['RECIPE']).split())

descTokenCounts= descTokenCounts.value_counts().to_dict()

df= df.drop_duplicates()


df= df.dropna()


def remove_non_ascii_1(text):

    return ''.join([i if ord(i) < 128 else ' ' for i in text])

def remove_non_ascii_2(text):

    return re.sub(r'[^\x20-\x7E]+',' ', text)

vocabSize=100000
tokens = sorted(descTokenCounts.items(),key=lambda x:x[1],reverse=True)[:vocabSize]

tokens = [i[0] for i in tokens]
tokens.append('token_unknown')
token_to_index = {}
index_to_token = {}


df['WHO'] = ([remove_non_ascii_1(sentence) for sentence in df['WHO']])
df['WHO'] = ([remove_non_ascii_2(sentence) for sentence in df['WHO']])


df['RECIPE']= ([remove_non_ascii_1(sentence) for sentence in df['RECIPE']])


# In[13]:




df['RECIPE']= ([remove_non_ascii_2(sentence) for sentence in df['RECIPE']])



In [203]:
model_name = 'allmy_jermaine_model1_jerm'


from sklearn.model_selection import train_test_split
#(X_train, X_test) = train_test_split(df2,test_size=0.20)


#with tf.device('/gpu:0'):
X_train, X_validation, X_test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])



with open('alltrain_data.json', 'wb') as f:
    f.write(X_train.to_json(orient='records', lines=True).encode('ascii', 'ignore'))

    
with open('alltest_data.json', 'wb') as g:
    g.write(X_test.to_json(orient='records', lines=True).encode('ascii', 'ignore'))


    
with open('allvalidation_data.json', 'wb') as t:
    t.write(X_validation.to_json(orient='records', lines=True).encode('ascii', 'ignore'))

def loadData(dataPath):
    data=[]
    start_time = time.time()
    with open(dataPath) as f:
        for line in f:
            data.append(line)
    loadTime = time.time() - start_time

    #print loadTime
    return data

training_data = loadData('alltrain_data.json')
validation_data = loadData('allvalidation_data.json') 
test_data = loadData('alltest_data.json') 


In [204]:
title_length= df['WHO'].map(len)
desc_length= df['RECIPE'].map(len)
titleCounts = df['WHO'].value_counts().to_dict()

charset=set(list(string.ascii_lowercase)+list(map(str,range(10)))+list("?$*!: @ ^ %- / <> ~ ` \\ [] {} \n = | \" _+#&-/\n %() ' '.;,"))
title_char_to_index_dict={c:i for i,c in enumerate(charset)}
title_index_to_char_dict={i:c for c,i in title_char_to_index_dict.items()}

In [205]:
print(title_char_to_index_dict)

{'?': 0, '9': 1, 'c': 2, 'h': 3, '3': 4, ')': 5, 'b': 6, 'd': 7, 'g': 8, '@': 9, '`': 10, '=': 11, '#': 12, '|': 13, '7': 14, ';': 15, 'n': 16, '*': 17, 'z': 18, '[': 19, 'l': 20, '4': 21, '}': 22, 'f': 23, 'v': 24, '6': 25, 'r': 26, '2': 27, ',': 28, '{': 29, '1': 30, 'm': 31, 'i': 32, 'k': 33, '<': 34, '/': 35, '~': 36, '$': 37, '^': 38, '_': 39, ':': 40, 'e': 41, '%': 42, ']': 43, 'y': 44, 'j': 45, 'u': 46, '8': 47, '.': 48, 'a': 49, ' ': 50, '!': 51, 'w': 52, 'x': 53, '>': 54, 'o': 55, 'p': 56, 't': 57, '+': 58, '&': 59, '\\': 60, '-': 61, '5': 62, 's': 63, '0': 64, '(': 65, "'": 66, 'q': 67, '"': 68, '\n': 69}


In [206]:



for i in range(len(tokens)):
    token_to_index[tokens[i]] = i+1
    index_to_token[i+1] = tokens[i] 


with open('allmodels_indices.json'.format(model_name),'w') as f:
    json.dump({'title_char_to_index_dict':title_char_to_index_dict,'title_index_to_char_dict':title_index_to_char_dict,              'token_to_index':token_to_index,'index_to_token':index_to_token},f)


    
title_max_len = np.ceil(np.percentile(title_length, title_length_percentile)).astype(int)

def title_char_tokenizer(entity,entity_max_len):
    preproced_entity=(" "*padding_length+entity)[:entity_max_len]
    padded_entity=preproced_entity+"".join([" "]*(entity_max_len-len(preproced_entity)))
    return [title_char_to_index_dict[i] for i in padded_entity]


## RECIPE tokenizer
## 0 padded in the end
desc_max_length = np.ceil(np.percentile(desc_length, desc_length_percentile)).astype(int)
def job_description_tokenizer(desc,entity_max_len):
    desc = re.sub("[,\/]",' ',desc).split()
    desc = [token_to_index[i] if i in token_to_index else token_to_index['token_unknown'] for i in desc]
    desc=(desc+[0]*entity_max_len)[:entity_max_len]
    return desc

In [207]:
with open ('allallmodels_model_params.json'.format(model_name),'w') as f:
    json.dump({'title_max_len':int(title_max_len),'desc_max_length':int(desc_max_length)},f)
    
## character embedding model for a list of titles
def char_embed_model_seq(input_layer,char_length,char_index_dict,entity_name,n_filters=512,embed_dim=embed_dim):
    '''
    Same as char_embed_model, but produces embeddings for a sequence of say, titles.
    Applies the same layers (as char_embed_model) to each member of the sequence
    '''    
    entity_onehot_layer=TimeDistributed(Lambda(lambda x: K.one_hot(x,len(char_index_dict)),output_shape=(char_length,len(char_index_dict))),name=entity_name+'_onehot_seq')(input_layer)
        
    conv7_layer=TimeDistributed(Conv1D(filters=n_filters,kernel_size=4,activation='relu'),name=entity_name+'_conv_7seq')(entity_onehot_layer)
    maxpool7_layer=TimeDistributed(GlobalMaxPool1D(),name=entity_name+'maxpool1seq')(conv7_layer)

    conv3_layer=TimeDistributed(Conv1D(filters=n_filters,kernel_size=2,activation='relu'),name=entity_name+'_conv_3seq')(entity_onehot_layer)
    maxpool3_layer=TimeDistributed(GlobalMaxPool1D(),name=entity_name+'maxpool2seq')(conv3_layer)

    pools_concat=keras.layers.concatenate([maxpool7_layer,maxpool3_layer],name=entity_name+'_maxpools_concatseq')
    #pools_dropped=(pools_concat)

    entity_dense_layer=TimeDistributed(Dense(n_filters*2,activation='relu'),name=entity_name+'_intermediate_denseseq')(pools_concat)
    
    dropout_layer=TimeDistributed(Dropout(0.4),name=entity_name+"_droppedseq")(entity_dense_layer)
    
    entity_final_dense=TimeDistributed(Dense(embed_dim),name=entity_name+'_last_denseseq')(dropout_layer)
    entity_prelu_layer=TimeDistributed(PReLU(),name=entity_name+'_preluseq')(entity_final_dense)
    embed_norm_layer=TimeDistributed(Lambda(lambda x: x/(K.epsilon()+K.sqrt(K.batch_dot(x,x,axes=-1)))),name=entity_name+'_unit_norm_embedseq')(entity_prelu_layer)

    entity_embed_model=keras.models.Model(input_layer,embed_norm_layer,name=entity_name+"_embedding_model")
    return entity_embed_model




# In[25]:


## job describtion embedding - average word embedding (remove 0 padding)
def desc_embed_model(input_layer,vocab_size,entity_name,embed_dim=embed_dim):
    wordEmbed = Embedding(input_dim = vocab_size,output_dim = embed_dim, name='word_embedding')(input_layer) 
    
    ## get average embedding for desc (remove 0 padding)
    mask=Lambda(lambda x: K.cast(K.minimum(1,K.cast(x,'int32')),'float32'),name='mask')(input_layer)
    mask_summed=Lambda(lambda x: 1/(K.epsilon()+K.sum(x,axis=-1)),name='mask_summed')(mask)
    mask_normed=keras.layers.multiply([mask,mask_summed],name='mask_normed')
    desc_embedding_averaged=keras.layers.dot([wordEmbed,mask_normed],axes=1,name='desc_embedding_averaged')
    
    desc_embed_model=Model(input_layer,desc_embedding_averaged,name=entity_name+"_embedding_model")
    return desc_embed_model
# In[26]:


alpha=0.5
sampling_titles = titleCounts.keys()

sampling_titles= [k for k in titleCounts.keys()]


# In[27]:


sampling_titles_array = np.array(sampling_titles)


# In[28]:


sampling_prob_array = np.array(list(titleCounts.values()))
sampling_prob_array = np.power(sampling_prob_array,alpha)
sampling_prob_array = sampling_prob_array/sum(sampling_prob_array)


# In[29]:


def sampling_based_on_similarity(title,sub_sampling_titles,number_of_negative_tokens=5):
## Randomly sample sub_sampling_titles based on emperical distribution
## Randomly sample titles from the selected sub-titles based on Levenshtein similarity
    st = time.time()
    sampling_titles = np.array(sub_sampling_titles)

    sampling_prob = np.array([Levenshtein.ratio(title,i) for i in sub_sampling_titles])#Levenshtein.ratio(title,i) for i in sub_sampling_titles])
    sampling_prob = sampling_prob/sum(sampling_prob)
    
    samples=np.random.choice(sampling_titles,size=number_of_negative_tokens,p=sampling_prob)
  
    return samples.tolist()



In [208]:
class empirical_sampling_gen(Sequence):

    def __init__(self, data, batch_size,number_of_negative_tokens,sampling_titles,sampling_prob,sub_sampling_size):
        '''        
        batch_size: number of resumes to process per batch        
        The actual number of training samples varies as each resume is splitted into sub-sequences
        '''
        self.batch_size = batch_size
        self.input_data = data
        self.data_length = len(data)
        self.number_negative_tokens = number_of_negative_tokens
        self.sampling_titles = sampling_titles
        self.sampling_prob = sampling_prob
        self.sub_sampling_size = sub_sampling_size
    def __len__(self):
        return int(np.ceil(self.data_length / float(self.batch_size)))
    

    def __getitem__(self, idx):
        i = idx
        batch_size = self.batch_size 
        desc_input_batch=[]
        title_input_batch=[]
        y_pos_batch=[]
        y_neg_batch=[] 
        titles = []
        for k in range(i * batch_size, min((i + 1) * batch_size, self.data_length)):
            line = json.loads(self.input_data[k])
            desc = job_description_tokenizer(line['RECIPE'],desc_max_length)
            #desc = str(desc)
            titles.append(line['WHO'])
            title = title_char_tokenizer(line['WHO'],title_max_len)
            #title= str(title)
            desc_input_batch.append(desc)
            title_input_batch.append([title]*self.number_negative_tokens)
            y_pos_batch.append(1)
            y_neg_batch.append([0]*self.number_negative_tokens)  
            
        ## Sample negative titles for each batch all together              
        neg_titles = np.random.choice(self.sampling_titles,                                                 size=len(titles)*self.number_negative_tokens,                                                 p=self.sampling_prob)
        
        ## Resample for titles with length below median title length based on Levenshtein similarity
        sub_sampling_titles = np.random.choice(self.sampling_titles,p=self.sampling_prob,                                               size=self.sub_sampling_size*len(titles),replace=True)
        for i,t in enumerate(titles):
            if np.random.random()>0.5:
                sub_titles = sub_sampling_titles[i*self.sub_sampling_size:(i+1)*self.sub_sampling_size]
                samples = sampling_based_on_similarity(t,sub_titles)
                neg_titles[i*self.number_negative_tokens:(i+1)*self.number_negative_tokens]=samples
        
        neg_title_input_batch = [title_char_tokenizer(t,title_max_len) for t in neg_titles]
        neg_title_input_batch = [neg_title_input_batch[x:x+self.number_negative_tokens]                                  for x in range(0, len(neg_title_input_batch), self.number_negative_tokens)]
        
        
             
        ## Convert inputs and output to numpy arrays
        inputs = list(map(np.array,[desc_input_batch,title_input_batch,neg_title_input_batch]))
        outputs = list(map(np.array,[y_pos_batch,y_neg_batch]))     
        #return (inputs, outputs,titles,neg_titles) 
        #print(inputs)
        return (inputs, outputs) 
    

    def on_epoch_end(self):
        #'Shuffle data after each epoch'
        np.random.shuffle(self.input_data)



# In[ ]:


# In[31]:


batch_size=80


# In[32]:


trainGen = empirical_sampling_gen(training_data,batch_size,number_of_negative_tokens,                                  sampling_titles_array,sampling_prob_array,sub_sampling_size=2200)                                 



#len(title_char_to_index_dict)


# In[33]:



testGen = empirical_sampling_gen(test_data,batch_size,number_of_negative_tokens,                                    sampling_titles_array,sampling_prob_array,sub_sampling_size=1500)


# In[34]:


validateGen = empirical_sampling_gen(validation_data,batch_size,number_of_negative_tokens,                                    sampling_titles_array,sampling_prob_array,sub_sampling_size=2200)                                 


# In[35]:


target_title_input=Input(shape=(number_of_negative_tokens,title_max_len), dtype='int32', name='targe_title_input')
neg_title_input = Input(shape=(number_of_negative_tokens,title_max_len), dtype='int32', name='ne_title_input')
title_embedding_model = char_embed_model_seq(target_title_input,title_max_len,title_char_to_index_dict,"titlee")
target_title_embed = title_embedding_model(target_title_input)
target_title_embed = Lambda(lambda x:x[:,0,:],name='title_embedding_take_first_elemente')(target_title_embed)
neg_title_embed = title_embedding_model(neg_title_input)



# In[36]:


## Get resume job description embedding
desc_input = Input(shape=(desc_max_length,),name='des_input')
desc_embedding_model = desc_embed_model(desc_input,vocabSize+2,'des')
desc_embed = desc_embedding_model(desc_input)

## Calcuate dot product and loss
desc_target_title_dot = keras.layers.dot([target_title_embed,desc_embed],axes=-1,name='des_target_dot')
pos_title_logit = Activation('sigmoid',name='po_title_logit')(desc_target_title_dot)

desc_neg_title_dot = keras.layers.dot([neg_title_embed,desc_embed],axes=-1,name='des_neg_dot')
neg_title_logit = Activation('sigmoid',name='ne_title_logit')(desc_neg_title_dot)


In [209]:
embeddings = desc_embedding_model.get_weights()[0]

In [210]:
np.set_printoptions(suppress=True)

In [211]:
words_embeddings = {w:embeddings[idx].flatten() for w, idx in descTokenCounts.items()}

In [212]:
   #count += 1

In [269]:
def recipe_emb(temp, words_embeddings):
    np.set_printoptions(suppress=True,
    formatter={'float_kind':'{:0.46f}'.format})
    count = 0
    v=1
    for w in temp.split():
    
        if w in words_embeddings:
            count +=1
            v-= ((words_embeddings[w]))
        #v= np.array(v)
    return (v)





In [270]:
#pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [259]:
df3= df[0:90000]

In [260]:
df3['embed_summed']=df3['RECIPE'].apply(lambda x: recipe_emb(x,words_embeddings))

/Users/jermainemarshall/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [261]:
df3.head(7)

,RECIPE,WHO,embed_summed
0,toastedsesameoil oliveoil nutbutter cidervinegar honey groundcayennepepper limewedges freshspinach spearmint springonions,two,"[1.1235876, 1.1093943, 1.0867372, 0.85236037, 0.89380324, 0.8650725, 0.98391545, 0.9461132, 0.9469015, 1.0970093, 0.8552572, 1.0715595, 0.9841607, 1.013585, 0.97388107, 0.9736649, 1.0155487, 0.99108434, 0.9708908, 0.97684455, 1.1266869, 1.1471821, 0.95085275, 0.87351066, 1.1188335, 1.1121885, 1.1366059, 0.9685107, 1.029752, 0.9673499, 0.85247856, 1.1385634, 0.9708395, 1.1972069, 0.83814836, 1.0772746, 0.9607101, 1.0581956, 0.96407497, 1.0055512, 0.92251134, 0.89204025, 0.9902143, 1.0689365, ..."
1,oliveoil cakeflour whiteonion choppedgarlic whitewine grapetomatoes chickenbouillon tomatopaste darkbrownsugar parsley,two,"[1.1576535, 1.0060086, 1.001102, 1.0257555, 0.965866, 1.0820988, 1.2114697, 1.0102463, 1.1409972, 0.9423051, 0.9791293, 1.1181381, 0.95848656, 0.8784251, 1.0353574, 0.9220266, 1.115515, 1.0678421, 1.2171474, 0.95343775, 1.0700219, 0.96431077, 0.96083665, 1.0005562, 1.1521571, 0.927994, 1.1126252, 0.93517196, 1.0194654, 0.9289029, 0.864626, 0.94313383, 0.9287649, 1.0493382, 0.7741933, 0.85782766, 0.89068496, 1.061048, 0.9233977, 0.98739916, 0.89586186, 0.9407691, 0.9297667, 1.0341085, 0.97049..."
2,fineseasalt crackedblackpepper choppedgarlic whiteonion ice freshbasil freshoregano freshthymeleaves saltedbutter shreddedparmesancheese,one,"[0.80973685, 0.86391956, 1.0316488, 1.0032533, 0.95308906, 0.93657243, 1.040899, 0.9817769, 1.0761225, 1.0651423, 1.0227959, 1.0026183, 1.1349108, 0.97062963, 1.2304887, 0.8398581, 1.022866, 1.0837965, 1.090834, 1.161133, 0.9819236, 1.0737371, 1.0018833, 0.95574737, 0.98670936, 0.82360303, 0.9653919, 0.8880768, 0.95349425, 1.0639567, 0.9214981, 1.0225074, 1.2548019, 1.1158266, 0.93849015, 0.896206, 0.9210321, 1.1299094, 0.94244236, 1.0328076, 0.9416909, 0.8554093, 0.85841954, 0.86125344, 0.8..."
3,unsaltedbutter shortening darkbrownsugar whitesugar hardboiledeggs extract cakeflour bakingsoda cinnamonsticks fineseasalt rolledoats goldenraisins,two,"[1.1159946, 1.0218182, 1.0490594, 1.1337346, 1.0819193, 1.0001538, 0.88156396, 1.0363054, 1.051347, 0.95685035, 0.97760034, 1.1465247, 1.0635517, 1.0436127, 1.0543605, 0.91675234, 1.0316187, 0.91868937, 1.026214, 1.0461168, 0.93609893, 0.9501556, 0.9354748, 1.0920302, 0.9907964, 1.0375928, 1.1761835, 0.9737048, 1.0834461, 0.8357477, 0.9849128, 0.8694957, 1.0597687, 1.0888761, 1.06421, 0.95366836, 0.8026761, 0.7020069, 0.8705663, 1.0084169, 0.84224504, 0.9776508, 1.014047, 0.905882, 0.9256118..."
4,ice fineseasalt saltedbutter cakeflour crackedblackpepper sourcream,two,"[1.0208696, 0.8956686, 0.9728144, 1.0291171, 1.0226023, 0.94862574, 0.99734336, 0.93378246, 1.0737027, 1.0024935, 1.0243517, 1.140763, 1.0883409, 0.9505637, 1.1412833, 0.91788536, 1.0551171, 0.9271432, 0.98769516, 1.0484502, 1.0087765, 1.0287459, 0.9814383, 1.0381069, 1.1644434, 0.9881387, 0.9279215, 0.96617115, 0.9829312, 0.9840515, 1.0381478, 0.9458911, 1.1932281, 1.0924323, 0.99459827, 1.0041562, 0.95646423, 1.0044742, 1.0243413, 0.9750632, 1.0100434, 0.8849414, 0.9189506, 0.9548748, 0.85..."
5,saltedbutter sweetpotatoes pineapple puremaplesyrup cinnamonsticks fineseasalt choppedwalnuts,three,"[0.94134396, 0.9448274, 1.1270329, 1.0474855, 1.0135624, 0.9859547, 1.0360261, 1.0584407, 0.9569531, 0.9783981, 1.0456549, 0.96260124, 1.0520508, 0.9978982, 0.9971763, 1.0118853, 0.96883535, 1.0734411, 0.8983274, 0.89759463, 1.1181602, 0.952811, 1.0497954, 0.8719612, 0.97365314, 0.95314246, 0.94482756, 1.1465884, 0.97682005, 0.93854696, 0.94228876, 1.1158723, 0.9636974, 0.97239214, 0.91030145, 0.9083359, 0.8979536, 0.9750195, 0.9319171, 1.1035287, 1.0131791, 0.92877036, 0.9360023, 0.98186535..."
6,hamburgerbuns pastasauce,two,"[0.95170057, 0.97133154, 1.032432, 0.99357784, 1.0231296, 1.0267057, 0.96467626, 0.97659945, 1.031161, 0.9527002, 0.9483558, 0.955703, 1.016209, 1.0348122, 0.97074485, 1.0066906, 1.0790608, 0.96130866, 1.0

In [262]:
df3.embed_summed= df3['embed_summed'].astype(str)

/Users/jermainemarshall/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [263]:
df3.embed_summed.str.replace('[','')
df3['embed_summed'] = df3['embed_summed'].str.replace('[', '')
df3['embed_summed'] = df3['embed_summed'].str.replace(']', '')
df3['embed_summed'] = df3['embed_summed'].str.replace('[\n\s]+', ',')
df3['embed_summed'] = df3['embed_summed'].str.replace(r'^,', '')

/Users/jermainemarshall/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/jermainemarshall/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/jermainemarshall/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [264]:
df4 = df3['embed_summed'].apply(lambda x: pd.Series(x.split(',')))

In [265]:
df3= df3[['RECIPE','WHO']].copy()

In [266]:
df3= df3.join(df4)

In [267]:
df3.head(1)

,RECIPE,WHO,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
0,toastedsesameoil oliveoil nutbutter cidervinegar honey groundcayennepepper limewedges freshspinach spearmint springonions,two,1.1235876083374023437500000000000000000000000000,1.1093943119049072265625000000000000000000000000,1.0867371559143066406250000000000000000000000000,0.8523603677749633789062500000000000000000000000,0.8938032388687133789062500000000000000000000000,0.8650724887847900390625000000000000000000000000,0.9839154481887817382812500000000000000000000000,0.9461132287979125976562500000000000000000000000,...,1.0437570810317993164062500000000000000000000000,0.9487783312797546386718750000000000000000000000,0.9575373530387878417968750000000000000000000000,0.8360086083412170410156250000000000000000000000,0.9395383000373840332031250000000000000000000000,0.9876493811607360839843750000000000000000000000,0.7833659648895263671875000000000000000000000000,1.0034283399581909179687500000000000000000000000,1.1035661697387695312500000000000000000000000000,1.0117907524108886718750000000000000000000000000


In [268]:
df3.to_csv('Embeddings_files_no_amounts/subtract_yummly_embeddings_no_counts.csv')

In [ ]:
for w in df2['RECIPE'][0].split():
    
    print(w)

In [ ]:
def word_embed_p(wordstr):
    sum = 0
    
    if word in embedding_dict:
        for i,word in embedding_dict.items():
            sum+=i[word]
    return sum
            
    
    

In [ ]:
words_embeddings['soup'].flatten()

In [ ]:
original_model=Model(inputs=[desc_input,target_title_input,neg_title_input],outputs=[pos_title_logit,neg_title_logit])


original_model.compile(optimizer='adam',metrics=['accuracy'],
                       loss=['binary_crossentropy','binary_crossentropy'],loss_weights=[1.,1.])                    

In [ ]:
original_model.fit_generator(trainGen,verbose=1,validation_data=validateGen,epochs=5, steps_per_epoch= 30,
                             workers=n_workers,max_queue_size=queue_size,use_multiprocessing=False,shuffle=True)


In [ ]:
open('all_recipe_model.json'.format(model_name), 'w').write(original_model.to_json())


# In[ ]:


original_model.save_weights('all_recipe_model.hdf5'.format(name=model_name))